<hr style="height: 1px;">
<i>This notebook was authored by the 8.S50x Course Team, Copyright 2022 MIT All Rights Reserved.</i>
<hr style="height: 1px;">
<br>

<h1>Guided Problem Set 15: Maxwell-Boltzmann Distribution from Molecular Simulation</h1>


<a name='section_15_0'></a>
<hr style="height: 1px;">


## <h2 style="border:1px; border-style:solid; padding: 0.25em; color: #FFFFFF; background-color: #90409C">P15.0 Overview</h2>


<h3>Navigation</h3>

<table style="width:100%">
    <tr>
        <td style="text-align: left; vertical-align: top; font-size: 10pt;"><a href="#section_15_1">P15.1 Modeling Collisions with a Boundary</a></td>
        <td style="text-align: left; vertical-align: top; font-size: 10pt;"><a href="#problems_15_1">P15.1 Problems</a></td>
    </tr>
    <tr>
        <td style="text-align: left; vertical-align: top; font-size: 10pt;"><a href="#section_15_2">P15.2 Modeling Interparticle Collisions</a></td>
        <td style="text-align: left; vertical-align: top; font-size: 10pt;"><a href="#problems_15_2">P15.2 Problems</a></td>
    </tr>
    <tr>
        <td style="text-align: left; vertical-align: top; font-size: 10pt;"><a href="#section_15_3">P15.3 Comparing Simulation to Theory</a></td>
        <td style="text-align: left; vertical-align: top; font-size: 10pt;"><a href="#problems_15_3">P15.3 Problems</a></td>
    </tr>
</table>



In [ ]:
#>>>RUN: P15.0-runcell00

#install the following:

#!pip install imageio
#!pip3 install torch torchvision torchaudio
#!pip install lmfit

In [ ]:
#>>>RUN: P15.0-runcell01

import itertools
from IPython.display import HTML
from scipy.integrate import odeint
import matplotlib.patches as patches
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import animation
from matplotlib.animation import PillowWriter
import imageio
from IPython.display import Image
import random

In [ ]:
#>>>RUN: P15.0-runcell02

#set plot resolution
%config InlineBackend.figure_format = 'retina'

#set default figure parameters
plt.rcParams['figure.figsize'] = (9,6)

medium_size = 12
large_size = 15

plt.rc('font', size=medium_size)          # default text sizes
plt.rc('xtick', labelsize=medium_size)    # xtick labels
plt.rc('ytick', labelsize=medium_size)    # ytick labels
plt.rc('legend', fontsize=medium_size)    # legend
plt.rc('axes', titlesize=large_size)      # axes title
plt.rc('axes', labelsize=large_size)      # x and y labels
plt.rc('figure', titlesize=large_size)    # figure title


<a name='section_15_1'></a>
<hr style="height: 1px;">

## <h2 style="border:1px; border-style:solid; padding: 0.25em; color: #FFFFFF; background-color: #90409C">P15.1 Modeling Collisions with a Boundary</h2>    

| [Top](#section_15_0) | [Previous Section](#section_15_0) | [Problems](#problems_15_1) | [Next Section](#section_15_2) |


<a name='problems_15_1'></a>     

| [Top](#section_15_0) | [Restart Section](#section_15_1) | [Next Section](#section_15_2) |

### <span style="border:3px; border-style:solid; padding: 0.15em; border-color: #90409C; color: #90409C;">Problem 15.1.1</span>

We know that for elastic collisions of a particle with a wall (stationary infinite mass object), the particle's velocity component perpendicular to the wall flips direction, while the parallel component stays the same.

<p align="center">
<img alt="Wall Collision" src="https://github.com/SangeonPark/psets/blob/main/pset4/images/wall_collision.png?raw=1" width="300"/>
</p>

$$v_{f, \perp} = -v_{i,\perp} $$

$$v_{f, \parallel} = v_{i,\parallel} $$


Complete the function `wall_collision` in the code below. This function takes as input `pos` and `vel` arrays (both with shape `[n_particles, 2]`) that represent position and velocity vectors of the particles, and returns the velocity vectors after collisions with the four sides of the wall.

Assume that the 2D box is a square with side length `L` equal to 1.

In [ ]:
#>>>PROBLEM: P15.1.1

def wall_collision(pos, vel):

    #collision with the right wall
    vel[pos[:, 0] > 1, 0] = #Your code goes here

    #collision with the left wall
    vel[pos[:, 0] < 0, 0] = #Your code goes here

    #collision with the top wall
    vel[pos[:, 1] > 1, 1] = #Your code goes here

    #collision with the bottom wall
    vel[pos[:, 1] < 0, 1] = #Your code goes here
    return vel


#-----------------------------------------------------
# CHECK YOUR RESULT

# Sample positions (x, y) of particles
pos_initial = np.array([
    [0.5, 0.5],  # Inside the box
    [1.5, 0.5],  # Colliding with the right wall
    [-0.5, 0.5], # Colliding with the left wall
    [0.5, 1.5],  # Colliding with the top wall
    [0.5, -0.5]  # Colliding with the bottom wall
])

# Sample velocities (vx, vy) of particles
vel_initial = np.array([
    [0.1, 0.1],  # Random velocity
    [0.2, 0.2],  # Random velocity
    [-0.3, -0.3],# Random velocity
    [0.4, 0.4],  # Random velocity
    [-0.5, -0.5] # Random velocity
])

print("Original Velocities:\n", vel_initial)

# Apply the wall_collision function to the sample data
updated_vel = wall_collision(pos_initial, vel_initial)

# Print the updated velocities after collision handling
print("\nUpdated Velocities after Collision Handling:\n", updated_vel)

### <span style="border:3px; border-style:solid; padding: 0.15em; border-color: #90409C; color: #90409C;">Problem 15.1.2</span>

Start with a simulation of a single particle bouncing off four walls of a 2D box, using the solution code for the previous question. The simulation will run in time steps `dt`, which is set to be a very small value.

First, in each time step, handle any wall collisions of the particle, flipping the appropriate component of the velocity vector using the function `wall_collision`.

Then, use the position update formula:

$$\vec{x}_{i+1} = \vec{x}_{i} + dt \cdot \vec{v}_{i+1}$$

which is discretizing the continuous relation

$$\vec{v} = \frac{d\vec{x}}{dt} $$

We save all the simulated position and velocity vectors in arrays of shape `[time_steps, n_particles ,2]`, where `n_particles` is equal to 1 in this case since we are simulating a single particle, and the last dimension denotes either the `x` or `y` component.

Check your result by printing the position and velocity of the particle in step `1111`.

In [ ]:
#>>>PROBLEM: P15.1.2

def simulate_wall(pos, vel, time_steps, dt):
    n_particles = pos.shape[0]
    pos_simulated = np.zeros((time_steps,n_particles, 2))
    vel_simulated = np.zeros((time_steps,n_particles, 2))

    pos_simulated[0] = pos
    vel_simulated[0] = vel

    #update the position and velocity
    for i in range(1,time_steps):
        #Your code here

    return pos_simulated, vel_simulated


#-----------------------------------------------------
# CHECK YOUR RESULT

pos_sim, vel_sim = simulate_wall(np.array([[0.5,0.5]]), np.array([[200.0,100.0]]), 8000, 0.0001)
print(pos_sim[1111][0], vel_sim[1111][0])

In [ ]:
#>>>RUN: P15.1-runcell01

#run the line below, if you have not previously
pos_sim, vel_sim = simulate_wall(np.array([[0.5,0.5]]), np.array([[200.0,100.0]]), 8000, 0.0001)

fig, ax = plt.subplots(1,1,figsize=(5,5))
ax.clear()
vmin = 0
vmax = 1
ax.set_xlim(0,1)
ax.set_ylim(0,1)
markersize = 2 * 0.005 * ax.get_window_extent().width  / (vmax-vmin) * 72./fig.dpi
red, = ax.plot([], [], 'o', color='red', markersize=markersize)

def animate(i):
    xred, yred = pos_sim[i][:,0], pos_sim[i][:,1]
    red.set_data(xred, yred)
    return [red]


ani = animation.FuncAnimation(fig, animate, frames=500, interval=50, blit=True)
HTML(ani.to_jshtml())

<a name='section_15_2'></a>
<hr style="height: 1px;">

## <h2 style="border:1px; border-style:solid; padding: 0.25em; color: #FFFFFF; background-color: #90409C">P15.2 Modeling Interparticle Collisions</h2>    

| [Top](#section_15_0) | [Previous Section](#section_15_1) | [Problems](#problems_15_2) | [Next Section](#section_15_3) |


<a name='problems_15_2'></a>     

| [Top](#section_15_0) | [Restart Section](#section_15_2) | [Next Section](#section_15_3) |

### <span style="border:3px; border-style:solid; padding: 0.15em; border-color: #90409C; color: #90409C;">Problem 15.2.1</span>

Implementing interparticle collisions means that, at each simulation step, we need to track which pairs of particles are close enough to collide. To do this, the particles must be given a size and then are assumed to collide if their centers are closer than a distance of one diameter apart. So, the collision detection starts with finding the distances between the centers of each pair of particles. If a pair is close enough, we will then simulate an elastic collision.

For bookkeeping indices, we give `n_particles` indices `0,1,...,n_particles-1`.

Given position vectors `position` of shape `[n_particles, 2]` and an array of indices corresponding to the particle pairs `idx_pair` of shape `[n_pair, 2]`, where each row is `[index_particle1, index_particle2]`, write a function that computes an array of shape `[n_pair]`, which is a pairwise distance of particles between every indexed pair.  

After running your code, enter the `distance_pairs` values for the `0`, `23`, `234`, and `444`th entries as a list of 4 numbers.

In [ ]:
#>>>PROBLEM: P15.2.1

# helper function that calculates distance between particles of given index pair
def compute_distance_pairs(position, idx_pair):
    delta_x = #Your code goes here
    delta_y = #Your code goes here
    distances = #np.sqrt(delta_x**2 + delta_y**2)
    return distances


#-----------------------------------------------------
# CHECK YOUR RESULT

np.random.seed(0x98a09fe)
n_particles = 50 #number of particles
pos_initial = np.random.rand(n_particles, 2)
indices = np.arange(0,n_particles,1)
idx_pair = np.array(list(itertools.combinations(indices,2)))
print(compute_distance_pairs(pos_initial, idx_pair)[[0,23,234,444]])

### <span style="border:3px; border-style:solid; padding: 0.15em; border-color: #90409C; color: #90409C;">Problem 15.2.2</span>

Now, given a distance pair array for a given array of index pairs, we need to calculate which pairs actually collide, meaning they satisfy the condition that the distance between their centers is smaller than `2*particle_radius`.

Write a simple function that takes `distance_pairs` as an input, and returns the array of indices of `idx_pairs` corresponding to pairs that collide. Hint: You can use a `np.where()` function!

To check your answer, use the same random array of particles that you previously defined, the code for which is again included below. **Enter the indices of all pairs that collide as a list of numbers.**


In [ ]:
#>>>PROBLEM: P15.2.2

def which_pair_collides(distance_pairs):
    #Your code goes here
    pass


#-----------------------------------------------------
# CHECK YOUR RESULT

np.random.seed(0x98a09fe)
n_particles = 50 #number of particles
pos_initial = np.random.rand(n_particles, 2)
indices = np.arange(0,n_particles,1)
idx_pair = np.array(list(itertools.combinations(indices,2)))

particle_radius = 0.01
distance_pairs = compute_distance_pairs(pos_initial, idx_pair)
which_pair_collides(distance_pairs)

### <span style="border:3px; border-style:solid; padding: 0.15em; border-color: #90409C; color: #90409C;">Problem 15.2.3</span>

After each collision event, what is the final velocity of the two particles?

Complete the code for `pair_collision` below, which takes `pos_1i`, `pos_2i`, `vel_1i` and `vel_2i` as inputs, representing the position and velocity vectors of the particles, respectively, and returns the velocity vectors after the collision.

Check your answer by printing the initial and final velocities for the two particles in the two pairs.

In [ ]:
#>>>PROBLEM: P15.2.3

def pair_collision(pos_1i, pos_2i, vel_1i, vel_2i):
    vel_1f = 0 #YOUR CODE HERE
    vel_2f = 0 #YOUR CODE HERE
    return vel_1f, vel_2f


#-----------------------------------------------------
# CHECK YOUR RESULT

# Sample initial positions (x, y) and velocities (vx, vy) of two pairs of particles
pos_1i = np.array([[0.5, 0.5],
                   [0.8, 0.8]])
pos_2i = np.array([[0.7, 0.6],
                   [0.4, 0.4]])

vel_1i = np.array([[0.1, 0.1],
                   [-0.1, -0.1]])
vel_2i = np.array([[-0.1, 0.1],
                   [0.1, -0.1]])


# Apply the pair_collision function to the sample data
vel_1f, vel_2f = pair_collision(pos_1i, pos_2i, vel_1i, vel_2i)

# Print the updated velocities after collision handling
print("Initial Velocities:")
print("Particle 1:", vel_1i)
print("Particle 2:", vel_2i)

print("\nUpdated Velocities after Collision Handling:")
print("Particle 1:", vel_1f)
print("Particle 2:", vel_2f)

### <span style="border:3px; border-style:solid; padding: 0.15em; border-color: #90409C; color: #90409C;">Problem 15.2.4</span>

Expand the `simulate` function you coded earlier to incorporate all particle-wise collisions.

The `idx_pair` variable keeps index pairs of every possible pair of particles.

Compute any pairwise collisions as the first action in each loop, followed by any wall collisions.

Complete the function!

In [ ]:
#>>>PROBLEM: P15.2.4

def simulate(pos, vel, time_steps, dt):

    nparticles = pos.shape[0]
    indices = np.arange(0,n_particles,1)
    idx_pair = np.array(list(itertools.combinations(indices,2)))

    pos_simulated = np.zeros((time_steps, n_particles, 2))
    vel_simulated = np.zeros((time_steps, n_particles, 2))

    pos_simulated[0] = pos
    vel_simulated[0] = vel

    #update the position and velocity based on particle collisions
    #and wall collisions
    for i in range(1,time_steps):
        #Your code here

    return pos_simulated, vel_simulated

In [ ]:
#>>>RUN: P15.2-runcell01

np.random.seed(4150032420)
n_particles = 65 #number of particles
particle_radius = 0.003 #radius of individual particles
L = 1 #side length of the box

pos_initial = np.random.rand(n_particles, 2) #inditial position of particles
# Setting up the initial 2D velocities of particles
vel_initial = np.zeros((n_particles,2))
vel_initial[pos_initial[:,0] >=L/2, 0] = -100
vel_initial[pos_initial[:,0] < L/2, 0] = 100
vel_initial[:, 1] = 0

pos_sim, vel_sim = simulate(pos_initial, vel_initial, 5000, 0.0001)

In [ ]:
#>>>RUN: P15.2-runcell02

fig, ax = plt.subplots(1,1,figsize=(5,5))
ax.clear()
vmin = 0
vmax = 1
ax.set_xlim(0,1)
ax.set_ylim(0,1)
markersize = 2 * particle_radius * ax.get_window_extent().width  / (vmax-vmin) * 72./fig.dpi
red, = ax.plot([], [], 'o', color='red', markersize=markersize)

def animate(i):
    xred, yred = pos_sim[i][:,0], pos_sim[i][:,1]
    red.set_data(xred, yred)
    return [red]



ani = animation.FuncAnimation(fig, animate, frames=500, interval=50, blit=True)
HTML(ani.to_jshtml())

<a name='section_15_3'></a>
<hr style="height: 1px;">

## <h2 style="border:1px; border-style:solid; padding: 0.25em; color: #FFFFFF; background-color: #90409C">P15.3 Comparing Simulation to Thermal Equilibrium Theory</h2>    

| [Top](#section_15_0) | [Previous Section](#section_15_2) | [Problems](#problems_15_3) | 

<a name='problems_15_3'></a>     

| [Top](#section_15_0) | [Restart Section](#section_15_3) |

### <span style="border:3px; border-style:solid; padding: 0.15em; border-color: #90409C; color: #90409C;">Problem 15.3.1</span>

In our simple model, we only have kinetic energy which is divided into 2 degrees of freedom, so:

$$d.o.f. \times \frac{1}{2}k_B T = 2 \times \frac{1}{2}k_B T = KE_{avg} = \frac{1}{2}m\left<{v}^2\right> \implies \boxed{\frac{m}{k_{B}T} = \frac{2}{\left<{v}^2\right>}}
$$

where $\left<v^2\right>$ denotes the average of the velocity squared (not to be confused with the square of the average velocity).

Of course, the energy of all of the particles are not exactly equal, only the average kinetic energy is given by the Equipartition Theorem. In this case, the probability distribution of the velocity is given by the Maxwell-Boltzmann distribution in 2 dimensions:

$$f(v) = \frac{m}{k_{B}T} v \exp\left(-\frac{m}{k_{B}T}\frac{v^2}{2} \right)$$

We can check if our system is reaching this theoretical thermal prediction by comparing the final simulated velocity distribution of the particles to this formula.

Set the initial velocity of the particles to $100~m/s$ and use this value to calculate ${m}/{k_{B}T}$. Notice that you do not need to determine  $m$, $k_B$ or $T$, since only this one combination of the three values appears in $f(v)$. Complete the function `maxwell_boltzmann(v)` with this information. To check the accuracy of your code, print the value of $f(v)$ for $v=100$.


In [ ]:
#>>>PROBLEM: P15.3.1

def maxwell_boltzmann(v):
    fv = #YOUR CODE HERE
    return fv


#-----------------------------------------------------
# CHECK YOUR RESULT

v_test = np.linspace(0, 2000, 1001)
print(maxwell_boltzmann(v_test)[np.where(v_test==100)[0]])

In [ ]:
#>>>RUN: P15.3-runcell01

#let's increase the number of particles significantly

np.random.seed(673720454)
n_particles = 400
particle_radius = 0.0002
v = np.linspace(0, 2000, 1001)

pos_initial = np.random.rand(n_particles, 2)
vel_initial = np.zeros((n_particles,2))
thetas = np.random.rand(n_particles)* 2 * np.pi
vel_initial[:, 0] = np.cos(thetas) * 100
vel_initial[:, 1] = np.sin(thetas) * 100

# This can take a while to run...
pos_sim, vel_sim = simulate(pos_initial, vel_initial, 200000, 0.0000008)

In [ ]:
#>>>RUN: P15.3-runcell02

bins = np.linspace(0,300,21)
plt.clf()
plt.figure()
plt.hist(np.sqrt(np.sum(vel_sim[-1]**2, axis=1)), bins=bins, density=True)
plt.plot(v,maxwell_boltzmann(v))
plt.xlabel('Velocity [m/s]')
plt.ylabel('# Particles')
plt.xlim(0,300)
#plt.show()

In [ ]:
#>>>RUN: P15.3-runcell03

fig, axes = plt.subplots(1, 2, figsize=(9,4))
axes[0].clear()
vmin = 0
vmax = 1
axes[0].set_xlim(0,1)
axes[0].set_ylim(0,1)
axes[0].get_xaxis().set_visible(False)
axes[0].get_yaxis().set_visible(False)
markersize = 10*2 * particle_radius * axes[0].get_window_extent().width  / (vmax-vmin) * 72./fig.dpi
red, = axes[0].plot([], [], 'o', color='red', markersize=markersize)
n, bins, patches = axes[1].hist(np.sqrt(np.sum(vel_sim[0]**2, axis=1)), bins=bins, density=True)
axes[1].plot(v,maxwell_boltzmann(v))
axes[1].set_xlim(0,300)
axes[1].set_ylim(0,0.01)
axes[1].get_yaxis().set_visible(False)

def animate(i):
    xred, yred = pos_sim[i][:,0], pos_sim[i][:,1]
    red.set_data(xred, yred)
    hist, _ = np.histogram(np.sqrt(np.sum(vel_sim[i]**2, axis=1)), bins=bins, density=True)
    for i, patch in enumerate(patches):
        patch.set_height(hist[i])
    return [red]

from itertools import chain
ani_hist = animation.FuncAnimation(fig, animate, frames=range(0,200000,500), interval=50, blit=True)
#list(chain(range(100),range(100, 39500, 100), range(39500, 40000)))

HTML(ani_hist.to_jshtml())

<h3>Going Beyond: Other Energy Scales</h3>

Does this simulation work this well for all energy scales (i.e., initial velocity distributions)? Is energy always conserved? Try making a plot of total energy versus time. Consider what, if any, changes are needed when simulating a systems with starting velocities of $500~m/s$ or $1000~m/s$.

The following code plots the time evolution of both the average velocity and total energy. Rerun code cell `P15.3-runcell01` and then this one to investigate what happens for different initial velocities. If you want to look at the final velocity distribution, as well as its time dependence, compared to the theoretical prediction, you need to rerun the solution for `P15.3-runcell02` to redefine the Maxwell-Boltzmann distribution for the different initial energy.

Note that the Maxwell-Boltzmann velocity distribution has a long tail on the high end. As a result, in a system starting with the same velocity for all particles, the average velocity will drop over time. However, that high velocity tail means that the total energy, which depends on the sum of the velocities squared, does not have the same time dependence as the average velocity.